In [ ]:
!pip install -r requirements.txt

In [ ]:
from fastapi import FastAPI, UploadFile, File, Form, BackgroundTasks, HTTPException
import azure.cognitiveservices.speech as speechsdk
import os, uuid
from pathlib import Path
import aiofiles
import smtplib
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from huggingface_hub import login
from fastapi.middleware.cors import CORSMiddleware
import torch
import requests
import os
from dotenv import load_dotenv  

app = FastAPI()


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# Replace this with your Hugging Face token
hf_token = os.getenv("HF_API")

# Log in
login(token=hf_token)

adapter_model_id = "SleepyGorilla/gemma-meetingbank-it"
base_model_id = "google/gemma-3-1b-it"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Load PEFT adapter and apply it
model = PeftModel.from_pretrained(base_model, adapter_model_id)
print("Model Loaded")
system_message = """You are an expert meeting assistant. Given the transcript of a meeting, your task is to read and analyze it, then generate a structured summary that captures the essence of the discussion.

Your summary must be correct, clear, thorough, and categorized into the following sections:

Discussion Points/Agenda : A brief overview of the main topics discussed during the meeting.

Decisions Made: Any clear conclusions, approvals, or agreed-upon actions.

Action Items: A list of tasks assigned to individuals or teams, with deadlines if mentioned.
"""

def generate_summary(transcript_text: str) -> str:
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": transcript_text}
    ]

    encoded = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    )

    input_ids = encoded.to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            max_new_tokens=2048,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    generated_tokens = output[0][input_ids.shape[-1]:]
    summary = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return summary






@app.get("/")
def read_root():
    return {"message": "Hello from FastAPI"}

AZURE_SPEECH_KEY = os.getenv("AZURE_SPEECH_KEY")
AZURE_SPEECH_REGION = "centralindia"
EMAIL_SENDER = "your@email.com"
EMAIL_PASSWORD = "your_email_password"
subscription_key = os.getenv("AZURE_TRANSLATE_KEY")

def detect_language(text, subscription_key):
    endpoint = "https://api.cognitive.microsofttranslator.com"
    path = "/detect?api-version=3.0"
    url = endpoint + path
    print(subscription_key)
    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Ocp-Apim-Subscription-Region": "centralindia",
        "Content-Type": "application/json"
    }

    body = [{"text": text}]
    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        detected_lang = response.json()[0]['language']
        return detected_lang
    else:
        print("Language detection failed")
        return None

def translate(text, subscription_key):
    detected_lang = detect_language(text, subscription_key)

    if detected_lang == "en":
        print("Text is already in English. Skipping translation.")
        return text

    endpoint = "https://api.cognitive.microsofttranslator.com"
    path = "/translate"
    url = endpoint + path

    params = {
        "api-version": "3.0",
        "from": detected_lang,
        "to": "en"
    }

    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Ocp-Apim-Subscription-Region": "centralindia",
        "Content-Type": "application/json"
    }

    body = [{"text": text}]
    response = requests.post(url, params=params, headers=headers, json=body)

    if response.status_code == 200:
        translations = response.json()
        translated_text = translations[0]["translations"][0]["text"]
        print(f"Translated from {detected_lang} to English: {translated_text}")
        return translated_text
    else:
        print("Translation failed")
        return None

def transcribe_audio(file_path: str) -> str:
    speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SPEECH_REGION)
    audio_config = speechsdk.audio.AudioConfig(filename=file_path)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    else:
        raise HTTPException(status_code=500, detail="Speech recognition failed.")


@app.post("/upload/audio")
async def upload_audio(background_tasks: BackgroundTasks, file: UploadFile = File(...), email: str = Form(...)):
    temp_path = f"temp/{uuid.uuid4()}_{file.filename}"
    os.makedirs("temp", exist_ok=True)

    async with aiofiles.open(temp_path, "wb") as out_file:
        content = await file.read()
        await out_file.write(content)

    try:
        transcribed_text = transcribe_audio(temp_path)
        transcribed_text = translate(transcribed_text, subscription_key)
        if transcribed_text is None:
            raise HTTPException(status_code=500, detail="Translation failed.")
        result = generate_summary(transcribed_text)
       
        output_str = f"Transcription: {transcribed_text}\nModel Output: {result}"

        # background_tasks.add_task(send_email, email, output_str)
        return {"message": output_str}
    finally:
        os.remove(temp_path)


@app.post("/upload/text")
async def upload_text(background_tasks: BackgroundTasks, file: UploadFile = File(...), email: str = Form(...)):
    content = await file.read()
    text = content.decode("utf-8")
    text = translate(text, subscription_key)
    if text is None:
        raise HTTPException(status_code=500, detail="Translation failed.")
    result = generate_summary(text)
    output_str = f"Model Output: {result}"

    # background_tasks.add_task(send_email, email, output_str)
    return {"message": output_str}


2025-04-17 09:01:02.482260: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 09:01:02.516649: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-17 09:01:02.516691: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-17 09:01:02.516732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 09:01:02.526011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

Model Loaded


In [2]:
from pyngrok import ngrok

# Open a tunnel on the port where Uvicorn will run
ngrok.set_auth_token("2vql2MRAvxhvDYjLTZvJc7rf8df_6G4SwFuko5GuU1bG3xMZv")
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print("Public URL:", ngrok_tunnel.public_url)

Public URL: https://d6e8-184-105-162-170.ngrok-free.app


In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

# Launch the server inside notebook
uvicorn.run(app, port=8000)

INFO:     Started server process [2113]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     27.97.221.8:0 - "GET /upload/text HTTP/1.1" 405 Method Not Allowed
